### Content-Based Model using Doc2Vec and XGBoost

In [1]:
import pandas as pd
import numpy as np
import import_ipynb
import data_acquisition
import feature_engineering
import pickle
from tqdm import tqdm
tqdm.pandas()

from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec
import string
import logging
import random

from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.preprocessing import LabelEncoder

from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import metrics 
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot

import time

importing Jupyter notebook from data_acquisition.ipynb
importing Jupyter notebook from feature_engineering.ipynb


#### 1. Reading Data
1. Fetching data generated function calls from files "feature_engineering" and "data_acquistion". 
2. Fetching sentiment score from previously built file "Sentiment_Scoring"

In [2]:
#Datasets

ratings_train = pd.read_pickle("./data/ratings_train_5_years.pkl")
ratings_validation = pd.read_pickle("./data/ratings_validation_5_years.pkl")
ratings_test = pd.read_pickle("./data/ratings_test_5_years.pkl")
sentiment_score = pd.read_pickle("./data/ratings_with_sentiment_score.pkl")

In [81]:
ratings_recommendation=pd.read_pickle('./data/ratings_recommendation_list.pkl')

#### 2. Data Preprocessing

In [3]:
# 2.1 Mapping Sentiment scores
sentiment_score=sentiment_score[['review_id','text','compound']]

ratings_train=ratings_train.merge(sentiment_score, on='review_id', how='inner')
ratings_train['text'] = ratings_train['text'].str.lower()

ratings_validation=ratings_validation.merge(sentiment_score, on='review_id', how='inner')
ratings_validation['text'] = ratings_validation['text'].str.lower()

ratings_test=ratings_test.merge(sentiment_score, on='review_id', how='inner')
ratings_test['text'] = ratings_test['text'].str.lower()


In [4]:
# 2.2 Label Encoding of user_id and business_id

lb_make = LabelEncoder()

label_df=pd.read_pickle('./data/ratings_feature_set.pkl')
label_df_user=label_df[['user_id']]
label_df_user.drop_duplicates(keep='first',inplace=True)
label_df_business=label_df[['business_id']]
label_df_business.drop_duplicates(keep='first',inplace=True)

label_df_user["user_code"] = lb_make.fit_transform(label_df_user["user_id"])
label_df_business["business_code"] = lb_make.fit_transform(label_df_business["business_id"])


ratings_train=ratings_train.merge(label_df_user, on='user_id', how='left')
ratings_train=ratings_train.merge(label_df_business, on='business_id', how='left')
ratings_train.drop(['user_id','business_id'],axis=1, inplace=True)

ratings_validation=ratings_validation.merge(label_df_user, on=['user_id'], how='left')
ratings_validation=ratings_validation.merge(label_df_business, on=['business_id'], how='left')
ratings_validation.drop(['user_id','business_id'],axis=1, inplace=True)

ratings_test=ratings_test.merge(label_df_user, on=['user_id'], how='left')
ratings_test=ratings_test.merge(label_df_business, on=['business_id'], how='left')
ratings_test.drop(['user_id','business_id'],axis=1, inplace=True)


#Rearranging the columns
cols = ratings_train.columns.tolist()
cols = cols[-1:] + cols[:-1]
cols = cols[-1:] + cols[:-1]

ratings_train = ratings_train[cols] 
ratings_validation = ratings_validation[cols]
ratings_test = ratings_test[cols]

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
//ana

In [84]:
ratings_recommendation=ratings_recommendation.merge(label_df_user, on=['user_id'], how='left')
ratings_recommendation=ratings_recommendation.merge(label_df_business, on=['business_id'], how='left')
ratings_recommendation.drop(['user_id','business_id'],axis=1, inplace=True)
ratings_recommendation.head(1)

,user_code,business_code
0,27451,2344


In [86]:
# 2.3 Ensuring all rows contain reviews

print('Null values in Train data :', 
      (ratings_train['user_code'].isnull().values.any(), 
       ratings_train['business_code'].isnull().values.any()))

print('Null values in Validation data :', 
      (ratings_validation['user_code'].isnull().values.any(), 
       ratings_validation['business_code'].isnull().values.any()))

print('Null values in Test data :', 
      (ratings_test['user_code'].isnull().values.any(), 
       ratings_test['business_code'].isnull().values.any()))

print('Null values in Recommendations data :', 
      (ratings_recommendation['user_code'].isnull().values.any(), 
       ratings_recommendation['business_code'].isnull().values.any()))

Null values in Train data : (False, False)
Null values in Validation data : (False, False)
Null values in Test data : (False, False)
Null values in Recommendations data : (False, False)


In [19]:
#Size of datasets
print("Size of training data containing 1 year of data :",ratings_train.shape[0])
print("Size of validation data containing 1 year of data :",ratings_validation.shape[0])
print("Size of test data containing 1 year of data :",ratings_test.shape[0])

print("#Columns of training data containing 1 year of data :",ratings_train.shape[1])
print("#Columns of validation data containing 1 year of data :",ratings_validation.shape[1])
print("#Columnsof test data containing 1 year of data :",ratings_test.shape[1])

print('#Unique users in train data :', ratings_train.user_code.nunique())

Size of training data containing 1 year of data : 267777
Size of validation data containing 1 year of data : 63420
Size of test data containing 1 year of data : 31710
#Columns of training data containing 1 year of data : 31
#Columns of validation data containing 1 year of data : 31
#Columnsof test data containing 1 year of data : 31
#Unique users in train data : 31710


In [8]:
#Subsetted Data
#ratings_train=ratings_train.head(10)
#ratings_validation=ratings_validation.head(10)
#ratings_test=ratings_test.head(10)

#### 3. Doc2Vec Model for Generating Vector of Reviews

In [20]:
# 3.1 Preprocessing text data for Doc2vec
def text_preprocessing_train(label,text):
    
    #Remove punctuation
    table = str.maketrans(dict.fromkeys(string.punctuation))
    text_v2 = text.translate(table)  
    
    #Tagging and Tokenization
    sentence_x=TaggedDocument(words= text_v2.split(), tags=[label])
    
    return sentence_x

In [21]:
# 3.2 Building the vector space for textual training data 

def training_vec(df, epochs, dm_value, window_value, sample_value, negative_value,model_tracker):
    
    doc2vec_df=df
    
    sentences=pd.DataFrame()
    sentences['sentence_tag']=doc2vec_df.progress_apply(lambda x : text_preprocessing_train(x['review_id'],x['text']),axis=1)
    sentences=list(sentences['sentence_tag'])
    
    # training the Doc2vec model

    model = Doc2Vec(min_count=5, window=window_value, size=100, sample=sample_value, 
                    negative=negative_value, workers=7, dbow_words=0, 
                    dm =dm_value, dm_concat =1, alpha=0.025 , min_alpha=0.025) # use fixed learning rate
    model.build_vocab(sentences)

    max_epochs = epochs
    for epoch in tqdm(range(max_epochs)):
        #print('iteration {0}'.format(epoch))
        random.shuffle(sentences)
        model.train(sentences,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        model.alpha -= 0.002 # decrease the learning rate
        model.min_alpha = model.alpha # fix the learning rate, no decay

    model.save("./data/doc2vec_text_review_" + model_tracker + ".d2v")
    ##model=Doc2Vec.load("./data/doc2vec_text_review.d2v") 
    
    # building the training dataset for classification problem
    
    doc2vec_df['x']=doc2vec_df['review_id'].apply(lambda x : model.docvecs[x])
    vector_df = pd.DataFrame(doc2vec_df['x'].values.tolist())
    df_ml=pd.concat([doc2vec_df[:], vector_df[:]], axis=1)
    df_ml.drop(['x','text','date','review_id'],axis=1,inplace=True)

    cols = ['Restaurants','Food','Fast Food','Nightlife','American (Traditional)', 
            'Bars', 'Mexican','Sandwiches', 'Pizza', 'Burgers']
    df_ml[cols] = df_ml[cols].apply(pd.to_numeric, errors='coerce', axis=1)
    
    return model, df_ml

In [22]:
# 3.3 Building the vector space for textual validation/test data 

def val_test_df_gen(df,model):
    
    val_test_df=df.copy()
    val_test_vec = np.empty(shape=(0,100))

    for i in tqdm(range(df.shape[0])):
        
        text=df['text'][i]
        #Remove punctuation
        table = str.maketrans(dict.fromkeys(string.punctuation))
        text_v2 = text.translate(table)  
        
        #Tokenization
        token = text_v2.split()
        
        #Vectorization
        val_test_vec= np.vstack((val_test_vec,model.infer_vector(token,alpha=0.005,steps=10).reshape(1,100)))
        
    vector_df=pd.DataFrame(val_test_vec)    
    val_test_df_gen=pd.concat([val_test_df[:], vector_df[:]], axis=1)
    val_test_df_gen.drop(['text','date','review_id'],axis=1,inplace=True)
    
    cols = ['Restaurants','Food','Fast Food','Nightlife','American (Traditional)', 
            'Bars', 'Mexican','Sandwiches', 'Pizza', 'Burgers']
    val_test_df_gen[cols] = val_test_df_gen[cols].apply(pd.to_numeric, errors='coerce', axis=1)
   
        
    return val_test_df_gen

#### 4. Xgboost Model for Classifying Ratings

In [23]:
# 4.1 Data Preparation for Xgboost

def xgboost_data_prep (df1, df2):
    X_train=df1.drop(['rating'],axis=1)
    y_train=(df1['rating']-1)

    X_val_test=df2.drop(['rating'],axis=1)
    y_true=(df2['rating']-1)
    
    return X_train, X_val_test, y_train, y_true


In [24]:
# 4.2 Xgboost model for rating classification

def xgboost_model (X_train, X_val_test, y_train, y_true,
                   learning_rate=0.1, n_estimators=100, max_depth=5, gamma=0, 
                   subsample=0.8,colsample_bytree=0.8,verbose=True) :
    
    start_time = time.time()
    print("--- %s seconds ---" % start_time)

    xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42, 
                                  learning_rate= learning_rate, n_estimators=n_estimators, 
                                  max_depth=max_depth, gamma=gamma, 
                                  subsample=subsample, colsample_bytree=colsample_bytree, seed=27)

    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_val_test)
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return xgb_model,y_true,y_pred

#### 5. Model Evaluation

In [27]:
def multiclass_roc_auc_score(y_test, y_pred, average):
    lb = LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)


In [28]:
def evaluation (y_true,y_pred) :   
    result={}
    result['Accuracy_Score']=accuracy_score(y_true, y_pred)
    result['Recall_Score_Macro']=recall_score(y_true, y_pred, average='macro')
    result['Recall_Score_Mirco']=recall_score(y_true, y_pred, average='micro')
    result['Precision_Score_Macro']=precision_score(y_true, y_pred, average='macro')
    result['Precision_Score_Mirco']=precision_score(y_true, y_pred, average='micro')
    result['AUC_Score_Macro']=multiclass_roc_auc_score(y_true, y_pred, average='macro')
    result['AUC_Score_Micro']=multiclass_roc_auc_score(y_true, y_pred, average='micro')

    print(result) 
    return result

#### 6. Doc2vec Model : Hyperparameter Tuning and Review Vector Generation

Test and Control approach has been used for tuning the 3 hyperparameters listed below. Thus the initial setting of these hyperparameters were : dm=0, window_value=10 and negative_value=10


1. Tuning for the hyperparameter - __Distributed Memory__ (dm) which either takes on 1 or 0. 

In [29]:
#Function Calls for vectorization of reviews and training/test data creation

dm=[0,1]
result_dm={}

for i in tqdm(dm) :
    
    tag=str("dm_"+str(i))
    print(tag)
    
    doc2vec_model,train_df = training_vec(ratings_train,epochs=10,dm_value=i, 
                                          window_value=10, sample_value=10e-6, 
                                          negative_value=10, model_tracker=tag)
    validate_df=(val_test_df_gen(ratings_validation,doc2vec_model))
    
    #Xgboost Model for hyperparameter evaluation
    X_train, X_val_test, y_train, y_true = xgboost_data_prep(train_df, validate_df)
    xgb_model, y_true, y_pred = xgboost_model(X_train, X_val_test, y_train, y_true )
    result_dm[tag] = evaluation(y_true, y_pred)
    
    # save the model to disk
    filename = './data/xgb_'+ tag +'.xgb'
    pickle.dump(xgb_model, open(filename, 'wb'))
 

  0%|          | 0/267777 [00:00<?, ?it/s]

dm_0



100%|██████████| 267777/267777 [00:14<00:00, 17949.98it/s]//anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")

  0%|          | 0/10 [00:00<?, ?it/s]//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).

 15%|█▌        | 9658/63420 [00:14<02:05, 429.18it/s]


 30%|██▉       | 18919/63420 [00:44<03:25, 216.37it/s]


 39%|███▉      | 24708/63420 [01:58<04:00, 161.09it/s]


...

 99%|█████████▊| 62576/63420 [09:12<00:13, 60.90it/s]


100%|██████████| 63420/63420 [09:24<00:00, 112.28it/s]

--- 1576469620.121372 seconds ---
--- 1212.0734288692474 seconds ---


100%|██████████| 2/2 [1:34:47<00:00, 2679.29s/it]

{'Accuracy_Score': 0.5378744875433618, 'Recall_Score_Macro': 0.3633699447837132, 'Recall_Score_Mirco': 0.5378744875433618, 'Precision_Score_Macro': 0.46839747555879124, 'Precision_Score_Mirco': 0.5378744875433618, 'AUC_Score_Macro': 0.6057589668481318, 'AUC_Score_Micro': 0.711171554714601}


In [30]:
result_dm

{'dm_0': {'Accuracy_Score': 0.5414853358561967,
  'Recall_Score_Macro': 0.36608160892186753,
  'Recall_Score_Mirco': 0.5414853358561967,
  'Precision_Score_Macro': 0.46869496182358505,
  'Precision_Score_Mirco': 0.5414853358561967,
  'AUC_Score_Macro': 0.6085644532010888,
  'AUC_Score_Micro': 0.713428334910123},
 'dm_1': {'Accuracy_Score': 0.5378744875433618,
  'Recall_Score_Macro': 0.3633699447837132,
  'Recall_Score_Mirco': 0.5378744875433618,
  'Precision_Score_Macro': 0.46839747555879124,
  'Precision_Score_Mirco': 0.5378744875433618,
  'AUC_Score_Macro': 0.6057589668481318,
  'AUC_Score_Micro': 0.711171554714601}}

__Best Results__ : Since results are better for __dm=0__, we will choose that for this data.

2. Tuning for Hyperparameter : __Window Size__ [5,8,10]

As we already know the results of dm_0 having window size =10, will proceed with other two

In [31]:
#Function Calls for vectorization of reviews and training/test data creation

window_size=[5,8]
result_window={}

for i in tqdm(window_size):
   
    tag=str("window_"+str(i))
    print(tag)
    
    doc2vec_model,train_df = training_vec(ratings_train,epochs=10,dm_value=0, 
                                          window_value=i, sample_value=10e-6, 
                                          negative_value=10, model_tracker=tag)
    validate_df=(val_test_df_gen(ratings_validation,doc2vec_model))
    
    #Xgboost Model
    X_train, X_val_test, y_train, y_true = xgboost_data_prep(train_df, validate_df)
    xgb_model, y_true, y_pred = xgboost_model(X_train, X_val_test, y_train, y_true )
    result_window[tag] = evaluation(y_true, y_pred)
    # save the model to disk
    filename = './data/xgb_'+ tag +'.xgb'
    pickle.dump(xgb_model, open(filename, 'wb'))
 

  0%|          | 0/267777 [00:00<?, ?it/s]

window_5



100%|██████████| 267777/267777 [00:17<00:00, 15095.10it/s]//anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")



  0%|          | 0/10 [00:00<?, ?it/s]//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).

 21%|██        | 13100/63420 [00:28<03:01, 276.82it/s]


 32%|███▏      | 19986/63420 [00:57<03:28, 208.53it/s]


...

 97%|█████████▋| 61819/63420 [06:52<00:21, 76.06it/s]


100%|██████████| 63420/63420 [07:13<00:00, 146.38it/s]

--- 1576474846.3304782 seconds ---
--- 1244.6974318027496 seconds ---


100%|██████████| 2/2 [1:18:45<00:00, 2359.48s/it]

{'Accuracy_Score': 0.5508672343109429, 'Recall_Score_Macro': 0.3749193387232138, 'Recall_Score_Mirco': 0.5508672343109429, 'Precision_Score_Macro': 0.4744960596438611, 'Precision_Score_Mirco': 0.5508672343109429, 'AUC_Score_Macro': 0.6165969684820266, 'AUC_Score_Micro': 0.7192920214443393}


In [32]:
result_window['window_10']=result_dm['dm_0']
result_window

{'window_5': {'Accuracy_Score': 0.5549984232103438,
  'Recall_Score_Macro': 0.38139002301669145,
  'Recall_Score_Mirco': 0.5549984232103438,
  'Precision_Score_Macro': 0.4826955937496832,
  'Precision_Score_Mirco': 0.5549984232103438,
  'AUC_Score_Macro': 0.6206789454652777,
  'AUC_Score_Micro': 0.7218740145064648},
 'window_8': {'Accuracy_Score': 0.5508672343109429,
  'Recall_Score_Macro': 0.3749193387232138,
  'Recall_Score_Mirco': 0.5508672343109429,
  'Precision_Score_Macro': 0.4744960596438611,
  'Precision_Score_Mirco': 0.5508672343109429,
  'AUC_Score_Macro': 0.6165969684820266,
  'AUC_Score_Micro': 0.7192920214443393},
 'window_10': {'Accuracy_Score': 0.5414853358561967,
  'Recall_Score_Macro': 0.36608160892186753,
  'Recall_Score_Mirco': 0.5414853358561967,
  'Precision_Score_Macro': 0.46869496182358505,
  'Precision_Score_Mirco': 0.5414853358561967,
  'AUC_Score_Macro': 0.6085644532010888,
  'AUC_Score_Micro': 0.713428334910123}}

__Best Results__ : __Window size of 5__ works the best, though all are giving nearly the same results

3. Tuning the hyperparameter - __Negative__ value of [5,10,15]

In the last run for window size, the value of negative was set to 10 for all window sizes. Hence we already have the performance results for window size 5 (best param) with negative of 10. We would thus run it for the other two and compare the results at the end.

In [33]:
#Function Calls for vectorization of reviews and training/test data creation

negative_k=[5,15]
result_neg={}

for i in tqdm(negative_k):
   
    tag=str("negative_k"+str(i))
    print(tag)
    
    doc2vec_model,train_df = training_vec(ratings_train,epochs=10,dm_value=0, 
                                          window_value=5, sample_value=10e-6, 
                                          negative_value=i, model_tracker=tag)
    validate_df=(val_test_df_gen(ratings_validation,doc2vec_model))
    
    #Xgboost Model
    X_train, X_val_test, y_train, y_true = xgboost_data_prep(train_df, validate_df)
    xgb_model, y_true, y_pred = xgboost_model(X_train, X_val_test, y_train, y_true )
    result_neg[tag] = evaluation(y_true, y_pred)
    # save the model to disk
    filename = './data/xgb_'+ tag +'.xgb'
    pickle.dump(xgb_model, open(filename, 'wb'))

  0%|          | 0/267777 [00:00<?, ?it/s]

negative_k5



100%|██████████| 267777/267777 [00:15<00:00, 17758.37it/s]//anaconda3/lib/python3.7/site-packages/gensim/models/doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")

  0%|          | 0/10 [00:00<?, ?it/s]//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).

 15%|█▌        | 9545/63420 [00:14<02:12, 407.02it/s]


 29%|██▊       | 18087/63420 [00:43<03:14, 232.62it/s]


 38%|███▊      | 24336/63420 [01:13<03:23, 192.50it/s]


...

 99%|█████████▉| 62973/63420 [07:25<00:06, 65.93it/s]


100%|██████████| 63420/63420 [07:32<00:00, 140.24it/s]

--- 1576480304.557965 seconds ---
--- 1295.9231729507446 seconds ---


100%|██████████| 2/2 [1:21:25<00:00, 2432.34s/it]

{'Accuracy_Score': 0.5555345316934721, 'Recall_Score_Macro': 0.3831168038342516, 'Recall_Score_Mirco': 0.5555345316934721, 'Precision_Score_Macro': 0.4745178576473972, 'Precision_Score_Mirco': 0.5555345316934721, 'AUC_Score_Macro': 0.6224114305878901, 'AUC_Score_Micro': 0.7222090823084201}


In [34]:
result_neg['negative_k10']=result_window['window_5']
result_neg

{'negative_k5': {'Accuracy_Score': 0.5432355723746453,
  'Recall_Score_Macro': 0.3655183137112895,
  'Recall_Score_Mirco': 0.5432355723746453,
  'Precision_Score_Macro': 0.46289101280538125,
  'Precision_Score_Mirco': 0.5432355723746453,
  'AUC_Score_Macro': 0.6088748511866204,
  'AUC_Score_Micro': 0.7145222327341532},
 'negative_k15': {'Accuracy_Score': 0.5555345316934721,
  'Recall_Score_Macro': 0.3831168038342516,
  'Recall_Score_Mirco': 0.5555345316934721,
  'Precision_Score_Macro': 0.4745178576473972,
  'Precision_Score_Mirco': 0.5555345316934721,
  'AUC_Score_Macro': 0.6224114305878901,
  'AUC_Score_Micro': 0.7222090823084201},
 'negative_k10': {'Accuracy_Score': 0.5549984232103438,
  'Recall_Score_Macro': 0.38139002301669145,
  'Recall_Score_Mirco': 0.5549984232103438,
  'Precision_Score_Macro': 0.4826955937496832,
  'Precision_Score_Mirco': 0.5549984232103438,
  'AUC_Score_Macro': 0.6206789454652777,
  'AUC_Score_Micro': 0.7218740145064648}}

__Best Results__ : __Negative 15__

Thus, for the doc2vec model we are fixing the parameters to be window size 5, dm=0 and negative =15. The model "doc2vec_text_review_negative_k15.d2v." would thus be utilized downstream for tuning the hyperparameters of the Xgboost model. 

4. Building the vectors for reviews in the validation data and test data using the best parameters of the Doc2vec model which were finalized

In [37]:
#Loading the optimum Doc2vec model
model_doc2vec=Doc2Vec.load("./data/doc2vec_text_review_negative_k15.d2v") 
doc2vec_df=ratings_train.copy()

#Building the training dataset for classification problem

doc2vec_df['x']=doc2vec_df['review_id'].apply(lambda x : model_doc2vec.docvecs[x])
vector_df = pd.DataFrame(doc2vec_df['x'].values.tolist())
train_df=pd.concat([doc2vec_df[:], vector_df[:]], axis=1)
train_df.drop(['x','text','date','review_id'],axis=1,inplace=True)

cols = ['Restaurants','Food','Fast Food','Nightlife','American (Traditional)', 
        'Bars', 'Mexican','Sandwiches', 'Pizza', 'Burgers']
train_df[cols] = train_df[cols].apply(pd.to_numeric, errors='coerce', axis=1)

validate_df=(val_test_df_gen(ratings_validation,model_doc2vec))
test_df=(val_test_df_gen(ratings_test,model_doc2vec))

100%|██████████| 31710/31710 [02:16<00:00, 232.49it/s]


#### 7. Xgboost Model : Hyperparameter Tuning and Model Training

In [46]:
# preprocessing the dataset for the Xgboost model
X_train, X_val_test, y_train, y_true = xgboost_data_prep(train_df, validate_df)

1. Tuning the hyperparameter - __n_estimators__ value of [50,100,150]

In [48]:
# n_estimators

n_estimators = [50,100,150]
result_nestimators={}

for i in tqdm(n_estimators):
   
    tag=str("n_estimators"+str(i))
    print(tag)
    
    #Xgboost Model    
    xgb_model, y_true, y_pred= xgboost_model (X_train, X_val_test, y_train, y_true,
                                              learning_rate=0.1, n_estimators=i, max_depth=5, gamma=0, 
                                              subsample=0.8,colsample_bytree=0.8,verbose=10)
    result_nestimators[tag] = evaluation(y_true, y_pred)
    
    # save the model to disk
    filename = './data/xgb_'+ tag +'.xgb'
    pickle.dump(xgb_model, open(filename, 'wb'))
    
result_nestimators

  0%|          | 0/3 [00:00<?, ?it/s]

n_estimators50
--- 1576531792.523413 seconds ---
--- 659.0089430809021 seconds ---


 33%|███▎      | 1/3 [10:59<21:58, 659.25s/it]

{'Accuracy_Score': 0.5524597918637654, 'Recall_Score_Macro': 0.38351295050035256, 'Recall_Score_Mirco': 0.5524597918637654, 'Precision_Score_Macro': 0.4673858586084956, 'Precision_Score_Mirco': 0.5524597918637654, 'AUC_Score_Macro': 0.6216772951168619, 'AUC_Score_Micro': 0.7202873699148534}
n_estimators100
--- 1576532451.7755032 seconds ---
--- 1200.237373828888 seconds ---


 67%|██████▋   | 2/3 [30:59<13:41, 821.62s/it]

{'Accuracy_Score': 0.5555029959003469, 'Recall_Score_Macro': 0.3831647089962575, 'Recall_Score_Mirco': 0.5555029959003469, 'Precision_Score_Macro': 0.475117088380894, 'Precision_Score_Mirco': 0.5555029959003469, 'AUC_Score_Macro': 0.6224142441740066, 'AUC_Score_Micro': 0.7221893724377167}
n_estimators150
--- 1576533652.2546859 seconds ---
--- 1864.8851792812347 seconds ---


100%|██████████| 3/3 [1:02:04<00:00, 1134.68s/it]

{'Accuracy_Score': 0.5575528224534847, 'Recall_Score_Macro': 0.38436691046122007, 'Recall_Score_Mirco': 0.5575528224534847, 'Precision_Score_Macro': 0.4745427386644443, 'Precision_Score_Mirco': 0.5575528224534847, 'AUC_Score_Macro': 0.6236755178982433, 'AUC_Score_Micro': 0.723470514033428}


{'n_estimators50': {'Accuracy_Score': 0.5524597918637654,
  'Recall_Score_Macro': 0.38351295050035256,
  'Recall_Score_Mirco': 0.5524597918637654,
  'Precision_Score_Macro': 0.4673858586084956,
  'Precision_Score_Mirco': 0.5524597918637654,
  'AUC_Score_Macro': 0.6216772951168619,
  'AUC_Score_Micro': 0.7202873699148534},
 'n_estimators100': {'Accuracy_Score': 0.5555029959003469,
  'Recall_Score_Macro': 0.3831647089962575,
  'Recall_Score_Mirco': 0.5555029959003469,
  'Precision_Score_Macro': 0.475117088380894,
  'Precision_Score_Mirco': 0.5555029959003469,
  'AUC_Score_Macro': 0.6224142441740066,
  'AUC_Score_Micro': 0.7221893724377167},
 'n_estimators150': {'Accuracy_Score': 0.5575528224534847,
  'Recall_Score_Macro': 0.38436691046122007,
  'Recall_Score_Mirco': 0.5575528224534847,
  'Precision_Score_Macro': 0.4745427386644443,
  'Precision_Score_Mirco': 0.5575528224534847,
  'AUC_Score_Macro': 0.6236755178982433,
  'AUC_Score_Micro': 0.723470514033428}}

__Best Results__ : __n estimators 100__

2. Tuning the hyperparameter - __Max Depth__ value of [5,7,9]

Previously built for n estimators =100 and max depth 5.

In [50]:
# max_depth

max_depth = [7,9]
result_maxdepth={}

for i in tqdm(max_depth):
   
    tag=str("max_depth"+str(i))
    print(tag)
    
    #Xgboost Model
    xgb_model, y_true, y_pred= xgboost_model (X_train, X_val_test, y_train, y_true,
                                              learning_rate=0.1, n_estimators=100, max_depth=i, gamma=0, 
                                              subsample=0.8,colsample_bytree=0.8,verbose=10)
    result_maxdepth[tag] = evaluation(y_true, y_pred)
    
    # save the model to disk
    filename = './data/xgb_'+ tag +'.xgb'
    pickle.dump(xgb_model, open(filename, 'wb'))
 

result_maxdepth['max_depth5'] = result_nestimators['n_estimators100']
result_maxdepth

  0%|          | 0/2 [00:00<?, ?it/s]

max_depth7
--- 1576536599.120513 seconds ---
--- 1657.5494332313538 seconds ---


 50%|█████     | 1/2 [27:37<27:37, 1657.83s/it]

{'Accuracy_Score': 0.5598391674550615, 'Recall_Score_Macro': 0.3893778635483092, 'Recall_Score_Mirco': 0.5598391674550615, 'Precision_Score_Macro': 0.4779944433029771, 'Precision_Score_Mirco': 0.5598391674550615, 'AUC_Score_Macro': 0.6262592825930176, 'AUC_Score_Micro': 0.7248994796594135}
max_depth9
--- 1576538256.949298 seconds ---
--- 2650.6627480983734 seconds ---


100%|██████████| 2/2 [1:11:48<00:00, 1955.78s/it]

{'Accuracy_Score': 0.5632450331125828, 'Recall_Score_Macro': 0.39689731972247405, 'Recall_Score_Mirco': 0.5632450331125828, 'Precision_Score_Macro': 0.4863000085800048, 'Precision_Score_Mirco': 0.5632450331125828, 'AUC_Score_Macro': 0.6311828574380792, 'AUC_Score_Micro': 0.7270281456953641}


{'max_depth7': {'Accuracy_Score': 0.5598391674550615,
  'Recall_Score_Macro': 0.3893778635483092,
  'Recall_Score_Mirco': 0.5598391674550615,
  'Precision_Score_Macro': 0.4779944433029771,
  'Precision_Score_Mirco': 0.5598391674550615,
  'AUC_Score_Macro': 0.6262592825930176,
  'AUC_Score_Micro': 0.7248994796594135},
 'max_depth9': {'Accuracy_Score': 0.5632450331125828,
  'Recall_Score_Macro': 0.39689731972247405,
  'Recall_Score_Mirco': 0.5632450331125828,
  'Precision_Score_Macro': 0.4863000085800048,
  'Precision_Score_Mirco': 0.5632450331125828,
  'AUC_Score_Macro': 0.6311828574380792,
  'AUC_Score_Micro': 0.7270281456953641},
 'max_depth5': {'Accuracy_Score': 0.5555029959003469,
  'Recall_Score_Macro': 0.3831647089962575,
  'Recall_Score_Mirco': 0.5555029959003469,
  'Precision_Score_Macro': 0.475117088380894,
  'Precision_Score_Mirco': 0.5555029959003469,
  'AUC_Score_Macro': 0.6224142441740066,
  'AUC_Score_Micro': 0.7221893724377167}}

__Best Results__ : __Max Depth 9__

3. Tuning the hyperparameter - __Subsample__ value of [0.6, 0.7, 0.8

Already have the results for max depth 9 and subample 0.8. We would therefore tune for the other 2 values.

In [51]:
# sub_sample

sub_sample= [0.6, 0.7]
result_subsample={}

for i in tqdm(sub_sample):
   
    tag=str("sub_sample"+str(i))
    print(tag)
    
    #Xgboost Model
    xgb_model, y_true, y_pred= xgboost_model (X_train, X_val_test, y_train, y_true,
                                              learning_rate=0.1, n_estimators=100, max_depth=9, gamma=0, 
                                              subsample=i,colsample_bytree=0.8,verbose=10)
    result_subsample[tag] = evaluation(y_true, y_pred)
    
    # save the model to disk
    filename = './data/xgb_'+ tag +'.xgb'
    pickle.dump(xgb_model, open(filename, 'wb'))

result_subsample['sub_sample0.8']=result_maxdepth['max_depth9']
result_subsample

  0%|          | 0/2 [00:00<?, ?it/s]

sub_sample0.6
--- 1576542061.520103 seconds ---
--- 2185.7147397994995 seconds ---


 50%|█████     | 1/2 [36:26<36:26, 2186.03s/it]

{'Accuracy_Score': 0.5648375906654053, 'Recall_Score_Macro': 0.3983036660670235, 'Recall_Score_Mirco': 0.5648375906654053, 'Precision_Score_Macro': 0.4903177558725192, 'Precision_Score_Mirco': 0.5648375906654053, 'AUC_Score_Macro': 0.6318220964175852, 'AUC_Score_Micro': 0.7280234941658783}
sub_sample0.7
--- 1576544247.552759 seconds ---
--- 2692.891798019409 seconds ---


100%|██████████| 2/2 [1:21:19<00:00, 2338.19s/it]

{'Accuracy_Score': 0.5623304951119521, 'Recall_Score_Macro': 0.3945463117147514, 'Recall_Score_Mirco': 0.5623304951119521, 'Precision_Score_Macro': 0.48409684609409054, 'Precision_Score_Mirco': 0.5623304951119521, 'AUC_Score_Macro': 0.6294702064228641, 'AUC_Score_Micro': 0.72645655944497}


{'sub_sample0.6': {'Accuracy_Score': 0.5648375906654053,
  'Recall_Score_Macro': 0.3983036660670235,
  'Recall_Score_Mirco': 0.5648375906654053,
  'Precision_Score_Macro': 0.4903177558725192,
  'Precision_Score_Mirco': 0.5648375906654053,
  'AUC_Score_Macro': 0.6318220964175852,
  'AUC_Score_Micro': 0.7280234941658783},
 'sub_sample0.7': {'Accuracy_Score': 0.5623304951119521,
  'Recall_Score_Macro': 0.3945463117147514,
  'Recall_Score_Mirco': 0.5623304951119521,
  'Precision_Score_Macro': 0.48409684609409054,
  'Precision_Score_Mirco': 0.5623304951119521,
  'AUC_Score_Macro': 0.6294702064228641,
  'AUC_Score_Micro': 0.72645655944497},
 'sub_sample0.8': {'Accuracy_Score': 0.5632450331125828,
  'Recall_Score_Macro': 0.39689731972247405,
  'Recall_Score_Mirco': 0.5632450331125828,
  'Precision_Score_Macro': 0.4863000085800048,
  'Precision_Score_Mirco': 0.5632450331125828,
  'AUC_Score_Macro': 0.6311828574380792,
  'AUC_Score_Micro': 0.7270281456953641}}

__Best Results__ : __Sub Sample 0.6__

Therefore final model has n estimators= 100, max depth =9 and subsample =0.6

#### 8. Final Model Evaluation - Validation and Test Data

1. Training Data

In [52]:
xgboost_model_final = pickle.load(open('./data/xgb_sub_sample0.6.xgb', 'rb'))

In [53]:
y_pred = xgboost_model_final.predict(X_train)
result_validate=evaluation(y_train,y_pred)

{'Accuracy_Score': 0.8172733281797914, 'Recall_Score_Macro': 0.8052081935334584, 'Recall_Score_Mirco': 0.8172733281797914, 'Precision_Score_Macro': 0.8495819387225142, 'Precision_Score_Mirco': 0.8172733281797914, 'AUC_Score_Macro': 0.87559839048227, 'AUC_Score_Micro': 0.8857958301123696}


2. Validation Data

In [54]:
# Validation Data

y_pred = xgboost_model_final.predict(X_val_test)
result_validate=evaluation(y_true,y_pred)


{'Accuracy_Score': 0.5648375906654053, 'Recall_Score_Macro': 0.3983036660670235, 'Recall_Score_Mirco': 0.5648375906654053, 'Precision_Score_Macro': 0.4903177558725192, 'Precision_Score_Mirco': 0.5648375906654053, 'AUC_Score_Macro': 0.6318220964175852, 'AUC_Score_Micro': 0.7280234941658783}


2. Test Data

In [56]:
# Test Data
X_train, X_test, y_train, y_true = xgboost_data_prep(train_df, test_df)
y_pred = xgboost_model_final.predict(X_test)
result_1yr=evaluation(y_true,y_pred)

{'Accuracy_Score': 0.593125197098707, 'Recall_Score_Macro': 0.40232450477823223, 'Recall_Score_Mirco': 0.593125197098707, 'Precision_Score_Macro': 0.4877691863480564, 'Precision_Score_Mirco': 0.593125197098707, 'AUC_Score_Macro': 0.6368792391933156, 'AUC_Score_Micro': 0.7457032481866919}


#### 9. Generating Recommendation

In [67]:
#User feature sets

user_cols=['user_code','review_count_x','average_stars','review_count_norm', 'compliment_score',
           'friends_norm','elite_count','user_lifetime','compliment_count','fans_norm']

review_cols=[i for i in range(100)]
review_cols= ['user_code'] + review_cols

train_df_user_subset_1=train_df[user_cols]
train_df_user_subset_2=train_df[review_cols]

train_df_user_subset_1.drop_duplicates(keep='first',inplace=True)
train_df_user_subset_2=train_df_user_subset_2.groupby(['user_code']).mean()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [68]:
#Item feature sets

item_cols=['business_code','stars','review_count_y','total_hours','Restaurants','Food','Fast Food', 'Nightlife',
           'American (Traditional)','Bars','Mexican','Sandwiches','Pizza','Burgers','total_checkins',
           'age_of_business']

train_df_item_subset_1=train_df[item_cols]
train_df_item_subset_2=train_df[['business_code','compound']]

train_df_item_subset_1.drop_duplicates(keep='first',inplace=True)
train_df_item_subset_2=train_df_item_subset_2.groupby(['business_code']).mean()

item_feat_repo=train_df_item_subset_1.merge(train_df_item_subset_2, on='business_code', how='inner')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [76]:
# features for ratings_recommendation from user and item feature sets

recommendation_df=ratings_recommendation.merge(train_df_user_subset_1, on='user_code', how='left')
recommendation_df=recommendation_df.merge(item_feat_repo, on='business_code', how='left')
recommendation_df=recommendation_df.merge(train_df_user_subset_2, on='user_code', how='left')

In [94]:
def top_k_recommendations (df, k):
    
    y_pred = xgboost_model_final.predict(df)
    y_probab = xgboost_model_final.predict_proba(df)
    
    top_k_df=df[['user_code','business_code']]
    top_k_df=top_k_df.merge(label_df_user, on='user_code', how='left')
    top_k_df=top_k_df.merge(label_df_business, on ='business_code', how='left')
    top_k_df['recommended_score']=list(y_pred)
    top_k_df['recommended_score']=top_k_df['recommended_score']+1
    top_k_df.drop(['user_code', 'business_code'],axis=1, inplace=True)

    top_k_df_final = top_k_df.sort_values('recommended_score',ascending = False).groupby('user_id').head(10)
    top_k_df_final.reset_index(inplace=True)
    top_k_df_final.drop('index', axis=1, inplace=True)
    
    return top_k_df_final

In [95]:
top_recommendations = top_k_recommendations (recommendation_df,10)